In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#!pip install statsmodels
import statsmodels.api as sm

#%load_ext lab_black

pd.set_option("max_columns", 500)

In [57]:
df = pd.read_csv("df_norm_3_time_all_median.csv")

In [58]:
# new columnn for exp_pos_cell_rel
df["exp_pos_cell_rel"] = (
    df["experiment_foldername"]
    + "_"
    + df["Position_n"].astype(str)
    + "_"
    + df["relative_ID"].astype(str)
)
df

,Unnamed: 0,experiment_foldername,Position_n,frame_i,Cell_ID,relationship,relative_ID,cell_vol_fl,division_frame_i,emerg_frame_i,generation_num,_NG_3D_concentration_dataPrepBkgr_from_vol_fl_meanProj,_NG_3D_concentration_autoBkgr_from_vol_fl_meanProj,_mKate_3D_concentration_dataPrepBkgr_from_vol_fl_meanProj,_mKate_3D_concentration_autoBkgr_from_vol_fl_meanProj,exp_pos,exp_pos_cell,time,ng_norm_factor,mkate_norm_factor,_NG_3D_concentration_autoBkgr_from_vol_fl_meanProj_norm,_mKate_3D_concentration_autoBkgr_from_vol_fl_meanProj_norm,h,lifetime,time_start,time_end,time_alive,exp_pos_cell_rel
0,0,exp5,Position_1,0,4,mother,-1.0,192.422420,-1.0,-1.0,1.0,998.311792,995.254531,558.862457,563.957893,exp5_Position_1,exp5_Position_1_4,0.00,995.254531,563.957893,1.000000,1.000000,0.500000,8.00,0.00,8.0,0.00,exp5_Position_1_-1.0
1,1,exp5,Position_1,1,4,mother,-1.0,231.171137,-1.0,-1.0,1.0,920.249378,916.479753,523.141186,527.853218,exp5_Position_1,exp5_Position_1_4,0.25,916.479753,527.853218,1.000000,1.000000,0.500000,8.00,0.00,8.0,0.25,exp5_Position_1_-1.0
2,2,exp5,Position_1,2,4,mother,12.0,225.112696,-1.0,-1.0,1.0,962.851917,959.913706,552.620776,557.517794,exp5_Position_1,exp5_Position_1_4,0.50,1829.195543,918.273603,0.524774,0.607137,0.536382,8.00,0.00,8.0,0.50,exp5_Position_1_12.0
3,3,exp5,Position_1,2,12,bud,4.0,2.227694,-1.0,2.0,0.0,2709.892133,2698.477380,1260.004826,1279.029413,exp5_Position_1,exp5_Position_1_12,0.50,1829.195543,918.273603,1.475226,1.392863,0.485641,7.50,0.50,8.0,0.00,exp5_Position_1_4.0
4,4,exp5,Position_1,3,4,mother,12.0,239.015157,-1.0,-1.0,1.0,925.495601,922.641627,536.718702,542.426648,exp5_Position_1,exp5_Position_1_4,0.75,1403.174293,708.707292,0.657539,0.765375,0.537893,8.00,0.00,8.0,0.75,exp5_Position_1_12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11569,11569,exp1,Position_2,32,562,bud,185.0,11.247725,-1.0,31.0,0.0,55.661832,631.900982,18.195251,423.120059,exp1_Position_2,exp1_Position_2_562,8.00,338.454979,219.343176,1.867016,1.929032,0.508168,0.25,7.75,8.0,0.25,exp1_Position_2_185.0
11570,11570,exp1,Position_2,32,563,bud,289.0,9.213000,-1.0,31.0,0.0,-223.529719,423.195109,2.152131,456.607416,exp1_Position_2,exp1_Position_2_563,8.00,338.454979,219.343176,1.250373,2.081703,0.624747,0.25,7.75,8.0,0.25,exp1_Position_2_289.0
11571,11571,exp1,Position_2,32,564,bud,121.0,10.495331,-1.0,31.0,0.0,-519.229220,78.869222,222.752572,643.037963,exp1_Position_2,exp1_Position_2_564,8.00,338.454979,219.343176,0.233027,2.931652,0.926366,0.25,7.75,8.0,0.25,exp1_Position_2_121.0
11572,11572,exp1,Position_2,32,565,bud,298.0,11.262431,-1.0,31.0,0.0,573.958908,1153.977022,123.409998,530.990294,exp1_Position_2,exp1_Position_2_565,8.00,338.454979,219.343176,3.409544,2.420820,0.415209,0.25,7.75,8.0,0.25,exp1_Position_2_298.0


In [60]:
df["exp_pos_cell_rel"] = df["exp_pos_cell_rel"].replace('\.0$', '', regex=True)
print(df["exp_pos_cell_rel"].dtype)

object


# Lateral Buds:
# Exp1: 
      # Pos1: Cell_ID 5 gets mainly NG copies -> exp1_220622_Position_1_5
      # Pos2: Cell_ID 4 (from zyg.1) gets mainly NG -> exp1_220622_Position_2_4
      # Pos6: Cell_ID 531 gets mainly NG -> exp_pos_cell_531 -> exp1_220622_Position_6_531


# Exp4:
    # Pos1: Cell_ID 5 gets mainly mKate copies (it still inherits few NG copies, and it also passes few to its daughter) -> exp4_060223_Position_1_5
    # Pos4: Cell_ID 607 & 1131 (mainly NG) -> exp4_060223_Position_4_607 & exp4_060223_Position_4_1131
    # Pos5: no side daughter with almost homoplasmic state from the beginning 

# Exp5:
      # No Pos from exp5 has lateral buds which inherit mainly one haplotype.

In [4]:
# find the lateral buds based on their exp_pos_cell and mark them as L in the new column
# find all cells that have this cell (lateral bud) as their relationship = mother and exp_pos_cell_rel = the lateral bud

In [70]:
df_lat = df

In [71]:
# exclude all buds that are 'True' in column 'lateral_bud' 
lateral_buds = [
    "exp1_Position_1_5",
    "exp1_Position_2_4",
    "exp1_Position_6_531",
    "exp4_Position_1_5",
    "exp4_Position_4_1131",
    "exp4_Position_4_607",
]

df_lat["lateral_bud"] = np.isin(df_lat["exp_pos_cell"], lateral_buds)
df_lat["lateral_bud"].value_counts()

df_lat["from_lateral_bud"] = np.isin(df_lat["exp_pos_cell_rel"], lateral_buds)
df_lat["lateral_bud"].value_counts()

#df_lat = df[~df['lateral_bud'].isin([True])]

False    11477
True        97
Name: lateral_bud, dtype: int64

In [83]:
# Identify rows where 'from_lateral_bud' is True
from_lateral_bud_true_rows = df_lat[df_lat["from_lateral_bud"]]
from_lateral_bud_true_rows.exp_pos_cell.unique()

array(['exp4_Position_1_1', 'exp4_Position_1_13', 'exp4_Position_1_32',
       'exp4_Position_1_52', 'exp4_Position_1_96', 'exp4_Position_1_167',
       'exp4_Position_1_319', 'exp4_Position_4_1', 'exp4_Position_4_1144',
       'exp4_Position_4_1845', 'exp1_Position_1_1', 'exp1_Position_1_23',
       'exp1_Position_1_94', 'exp1_Position_1_233', 'exp1_Position_1_435',
       'exp1_Position_2_1', 'exp1_Position_2_16', 'exp1_Position_2_43',
       'exp1_Position_2_78', 'exp1_Position_2_125', 'exp1_Position_2_214',
       'exp1_Position_2_455'], dtype=object)

In [86]:
# Extract 'exp_pos_cell' values from the identified rows
exp_pos_cell_values = from_lateral_bud_true_rows.exp_pos_cell.unique()

# Find rows where 'exp_pos_cell_rel' matches the extracted 'exp_pos_cell' values
matching_rows = df_lat[df_lat["exp_pos_cell_rel"].isin(exp_pos_cell_values)]

# Mark the corresponding rows in 'from_lateral_bud_chain' as True
df_lat.loc[matching_rows.index, "from_lateral_bud_lineage"] = True

In [ ]:
# save csv to run the lineage_correlations script
df_norm_lat.to_csv("df_norm_lat.csv")

In [91]:
# Exclude cells that meet any of the exclusion criteria
df_nolat_buds = df_lat[~(df_lat["lateral_bud"] | df_lat["from_lateral_bud"] | df_lat["from_lateral_bud_lineage"])]

In [93]:
df_nolat_buds.to_csv("df_nolat_buds.csv")

In [94]:
df_lat["lateral_bud"].value_counts()

False    11477
True        97
Name: lateral_bud, dtype: int64

In [95]:
df_lat["from_lateral_bud"].value_counts()

False    11462
True       112
Name: from_lateral_bud, dtype: int64

In [96]:
df_lat["from_lateral_bud_lineage"].value_counts()

False    11223
True       351
Name: from_lateral_bud_lineage, dtype: int64

In [97]:
df_nolat_buds["from_lateral_bud_lineage"].value_counts()

False    11111
Name: from_lateral_bud_lineage, dtype: int64